In [1]:
import pandas as pd
import numpy as np
from pydataset import data

from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

 # 1 <br>
 Load the tips dataset.

In [2]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


# A <br>
Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [3]:
df['tip_percentage'] = df.tip / df.total_bill
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


# B <br>
Create a column named price_per_person. This should be the total bill divided by the party size.

In [4]:
# 'size' is a reserved word, so we have to call size in bracket notation
df['price_per_person'] = df['total_bill']/df['size']
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500


# C <br>
Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?

I believe that total_amount would be the best predictor of tip amount.
Size may be a good predictor of tip_percentage.

# D <br>
Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?

In [5]:
# Encode the data
df.replace({'sex':{'Male':1, 'Female':0}}, inplace=True)
df.replace({'smoker':{'Yes':1, 'No':0}}, inplace=True)
df.replace({'time':{'Lunch':0, 'Dinner':1}}, inplace=True)
df.replace({'day':{'Thur':0, 'Fri':1, 'Sat':3, 'Sun':4}}, inplace=True)
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,0,0,4,1,2,0.059447,8.495000
2,10.34,1.66,1,0,4,1,3,0.160542,3.446667
3,21.01,3.50,1,0,4,1,3,0.166587,7.003333
4,23.68,3.31,1,0,4,1,2,0.139780,11.840000
5,24.59,3.61,0,0,4,1,4,0.146808,6.147500


In [6]:
# Select K best

X = df.drop(columns=['tip', 'tip_percentage'])
y = df.tip

# Scaling should be done on train subset, so we need to split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Create the object
kbest = SelectKBest(f_regression, k=2)

# Fit the object
kbest.fit(X_train_scaled, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7fcf0a415940>)

In [8]:
# Use the object
kbest.get_support()

array([ True, False, False, False, False,  True, False])

In [9]:
X_train.columns[kbest.get_support()]

Index(['total_bill', 'size'], dtype='object')

In [10]:
# Select K best
# The best predictive features are 'total_bill' and 'size'

In [12]:
# Recursive feature elimination

# Create the object
rfe = RFE(estimator=LinearRegression(), n_features_to_select=2)

# Fit the object
rfe.fit(X_train_scaled, y_train)

# Use the object
rfe.get_support()

array([ True, False, False, False, False, False,  True])

In [13]:
X_train.columns[rfe.get_support()]

Index(['total_bill', 'price_per_person'], dtype='object')

In [14]:
pd.Series(rfe.ranking_, index=X_train.columns)

total_bill          1
sex                 5
smoker              6
day                 3
time                4
size                2
price_per_person    1
dtype: int64

In [15]:
# RFE
# The answer is different than Select K Best
# The top 2 predictive feaures are 'total_bill' and 'price_per_person'

# E <br>
Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?

In [16]:
# Setting up the problem

X = df.drop(columns='tip_percentage')
y = df.tip_percentage

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [17]:
# Select K Best 
kbest = SelectKBest(f_regression, k=2)
kbest.fit(X_train_scaled, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x7fcf0a415940>)

In [18]:
kbest.get_support()

array([False,  True, False, False, False, False, False,  True])

In [19]:
X_train.columns[kbest.get_support()]

Index(['tip', 'price_per_person'], dtype='object')

In [20]:
# Select K best
# The best predictive features are 'tip' and 'price_per_person'

In [21]:
# RFE
rfe = RFE(estimator=LinearRegression(), n_features_to_select=2)
rfe.fit(X_train_scaled, y_train)
rfe.get_support()

array([ True,  True, False, False, False, False, False, False])

In [22]:
X_train.columns[rfe.get_support()]

Index(['total_bill', 'tip'], dtype='object')

In [23]:
pd.Series(rfe.ranking_, index=X_train.columns)

total_bill          1
tip                 1
sex                 7
smoker              4
day                 5
time                6
size                2
price_per_person    3
dtype: int64

In [24]:
# RFE
# The best predictive features are 'total_bill' and 'tip'

# F <br>
Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

In [25]:
# K best looks at each feature indivually
# RFE looks at each feature as part of the whole, but has a higher computational cost

# 2 <br>
Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [27]:
def select_kbest(X, y, k):
    
    # Take in predictors and target.
    # Create train and test subsets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create the object
    kbest = SelectKBest(f_regression, k=k)
    
    # Fit the object
    kbest.fit(X_train_scaled, y_train)
    
    # Use the object
    top_k = X_train.columns[kbest.get_support()]
    
    return top_k

In [28]:
X = df.drop(columns='tip_percentage')
y = df.tip_percentage
k = 2

select_kbest(X, y, k)

Index(['tip', 'price_per_person'], dtype='object')

# 3 <br>
Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [29]:
def rfe(X, y, k):
    
    # Take in predictors and target 
    # Create train and test subsets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=123)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    rfe = RFE(estimator=LinearRegression(), n_features_to_select=k)
    rfe.fit(X_train_scaled, y_train)
    rfe.get_support()
    
    top_k = X_train.columns[rfe.get_support()]
    
    return top_k

In [30]:
X = df.drop(columns='tip_percentage')
y = df.tip_percentage
k = 2

rfe(X, y, k)

Index(['total_bill', 'tip'], dtype='object')

# 4 <br>
Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [31]:
import pydataset
df = pydataset.data('swiss')
df.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [32]:
X = df.drop(columns='Fertility')
y = df.Fertility
k = 3

In [33]:
select_kbest(X, y, k)

Index(['Examination', 'Education', 'Catholic'], dtype='object')

In [34]:
rfe(X, y, k)

Index(['Agriculture', 'Education', 'Catholic'], dtype='object')